In [4]:
import pandas as pd
import xml.etree.ElementTree as ET

In [149]:
xml_data = open('/home/daniel/Рабочий стол/pagerank.xml', 'r').read()  # Read file
root = ET.XML(xml_data)  # Parse XML

data = []
cols = []
for i, child in enumerate(root):
    source = []
    source.append([subchild.attrib.get('source') for subchild in child])
    data.append(source)
    target = []
    target.append([subchild.attrib.get('target') for subchild in child])
    data.append(target)
    value = []
    value.append([subchild.attrib.get('value') for subchild in child])
    data.append(value)

PADGM = pd.DataFrame()
PADGM['source'] = data[0][0]
PADGM['target'] = data[1][0]
PADGM['value'] = data[2][0]

PADGB = pd.DataFrame()
PADGB['source'] = data[3][0]
PADGB['target'] = data[4][0]
PADGB['value'] = data[5][0]

In [334]:
import numpy as np
import random

def get_index(obj, objects):
    i = 0
    while (objects[i] != obj):
        i+=1
    return i

def PageRank(table, eps = 1e-300, d = 0.85):
    global pageRank, nodes, outputcount, it
    new_pageRank = np.asarray(pageRank)
    visited = [0] * len(nodes)
    can_continue = True
    q = []
    while(can_continue):
        if not q:
            v = random.randrange(count)
        else:
            v = q.pop(0)
        visited[v] = 1
        next_objects = table.loc[(table['source'] == nodes[v]) & (table['value'] == '1.0000')]['target']
        prev_objects = table.loc[(table['target'] == nodes[v]) & (table['value'] == '1.0000')]['source']
        for obj in next_objects:
            i = np.where(nodes == obj)[0][0]
            if(visited[i] == 0):
                q.append(i)
        new_pageRank[v] = (1 - d)/len(nodes)
        for obj in prev_objects:
            i = np.where(nodes == obj)[0][0]
            new_pageRank[v]+= d * pageRank[i] / outputcount[i]
        if (sum(visited) == len(nodes)):
            can_continue = False
    it+=1
    if ((np.linalg.norm(np.asarray(new_pageRank) - np.asarray(pageRank)) < eps) and (it > 10)):
        print("Done", it, 'iterations')
    else:
        pageRank = new_pageRank
        PageRank(table)
    

# Смотрим на граф брачных связей

In [338]:
nodes = PADGM['source'].unique()
pageRank = [(1-d)/len(nodes)] * len(nodes)
outputcount = [0] * len(nodes)
for (i, node) in enumerate(nodes):
    outputcount[i] = len(PADGM.loc[(PADGM['source'] == nodes[i]) & (PADGM['value'] == '1.0000')])
it = 0
PageRank(PADGM)
df = pd.DataFrame(pageRank).T
df.columns = nodes
df

Done 11 iterations


,ACCIAIUOL,ALBIZZI,BARBADORI,BISCHERI,CASTELLAN,GINORI,GUADAGNI,LAMBERTES,MEDICI,PAZZI,PERUZZI,PUCCI,RIDOLFI,SALVIATI,STROZZI,TORNABUON
0,0.02856,0.073853,0.046944,0.064153,0.06462,0.030266,0.091725,0.028866,0.135954,0.033693,0.063305,0.009375,0.064784,0.057218,0.08209,0.066482


# Смотрим на граф бизнес связей

In [340]:
nodes = PADGB['source'].unique()
pageRank = [(1-d)/len(nodes)] * len(nodes)
outputcount = [0] * len(nodes)
for (i, node) in enumerate(nodes):
    outputcount[i] = len(PADGB.loc[(PADGB['source'] == nodes[i]) & (PADGB['value'] != '0.0000')])
it = 0
PageRank(PADGB)
df = pd.DataFrame(pageRank).T
df.columns = nodes
df

Done 11 iterations


,ACCIAIUOL,ALBIZZI,BARBADORI,BISCHERI,CASTELLAN,GINORI,GUADAGNI,LAMBERTES,MEDICI,PAZZI,PERUZZI,PUCCI,RIDOLFI,SALVIATI,STROZZI,TORNABUON
0,0.009375,0.009375,0.086903,0.062144,0.061898,0.049707,0.04403,0.080234,0.128623,0.031241,0.08003,0.009375,0.009375,0.031241,0.009375,0.031239
